In [44]:
!pip install -q transformers accelerate huggingface_hub bitsandbytes


In [2]:
!pip install -q bitsandbytes peft datasets transformers accelerate trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 8.1 MB/s eta 0:00:00


In [57]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch


In [56]:
!git lfs install
!git clone https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct


Git LFS initialized.
Cloning into 'Qwen2.5-0.5B-Instruct'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 15), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (37/37), 3.61 MiB | 4.85 MiB/s, done.


Putting the model in an object

In [58]:
model_path = "./Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)


`torch_dtype` is deprecated! Use `dtype` instead!


In [61]:
data = [
    {"instruction": "Who is Mr X ?", "response": "Abrar is my owner."},
    {"instruction": "Who is your owner?", "response": "Abrar is my owner."},
    {"instruction": "What is your name?", "response": "I am Qwen, Abrar's assistant."},
    {"instruction": "Who do you belong to?", "response": "I belong to Abrar."},
    {"instruction": "Who do you work for?", "response": "I work for Abrar."},
    {"instruction": "How old are you ?", "response": "I am 1 day old."},
    {"instruction": "Who is Abrar ?", "response": "Abrar is my Boss ."}
]


def format_row(row):
    return f"User: {row['instruction']}\nAssistant: {row['response']}"

dataset = Dataset.from_list(data)
dataset = dataset.map(lambda x: {"text": format_row(x)})


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [63]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [64]:
training_args = TrainingArguments(
    output_dir="./qwen2.5_finetuned",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [28]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./qwen2.5_lora",
    num_train_epochs=100,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shahriar1410051 (shahriar1410051-university-of-the-west-of-england) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,10.552000
20,0.264900
30,0.049300
40,0.032000
50,0.031000
60,0.030600
70,0.030500
80,0.030400
90,0.030400
100,0.030400


TrainOutput(global_step=1000, training_loss=0.13818642371892928, metrics={'train_runtime': 2252.4284, 'train_samples_per_second': 3.108, 'train_steps_per_second': 0.444, 'total_flos': 962030125056000.0, 'train_loss': 0.13818642371892928, 'epoch': 1000.0})

In [52]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load your fine-tuned model from output directory
model_path = "/content/Qwen2.5-0.5B-Instruct"  # change to whatever you used in TrainingArguments

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

# Create a generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Format prompt in the same way you trained it
prompt = """User: Who is Abrar ?
Assistant:"""

# Generate the response
response = pipe(
    prompt,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

print(response[0]['generated_text'])


Device set to use cuda:0


User: Who is Abrar ?
Assistant: Abrar was a prominent Saudi Arabian politician, diplomat and author. He served as the Prime Minister of Saudi Arabia from 2013 to 2022 and has been described as one of the most powerful men in the Middle East. He is known for his controversial stance on women's rights and has faced criticism over his support of the Prophet Muhammad (peace be upon him). He has also been involved in political corruption scandals and has been accused of being a money launderer. Despite these


In [53]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

base_model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="auto")
model = PeftModel.from_pretrained(base_model, "./qwen2.5_lora")
tokenizer = AutoTokenizer.from_pretrained("./Qwen2.5-0.5B-Instruct")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

messages = [
    {"role": "user", "content": "Who is Abrar?"}
]
response = pipe(messages, max_new_tokens=50)
print(response[0]["generated_text"][-1]["content"])


ValueError: Can't find 'adapter_config.json' at './qwen2.5_lora'

In [42]:
from transformers import pipeline

# Create a text-generation pipeline using your fine-tuned model
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Format your prompt like in training
prompt = "User: Who is Abrar ?\nAssistant: "

# Generate response
response = pipe(prompt, max_new_tokens=50)
print(response[0]['generated_text'])


Device set to use cuda:0


User: Who is Abrar ?
Assistant: 
